In [74]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import geodatasets
import numpy as np
import math
import matplotlib.pyplot as plt
from shapely.geometry import LineString
import matplotlib.pyplot as plt
from tqdm import trange
import pickle
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import os
import scipy.stats as stats
import pickle
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pandas as pd


In [78]:
with open("FINAL/LINEAR/LINEAR_MARKOV_NO_ADJUST_results_1mil_cooling_with_annealing.pkl", "rb") as file:
    markov_results, markov_temp, markov_all_runs_data_lin = pickle.load(file)

with open("FINAL/EXPO/EXPO_MARKOV_NO_ADJUST_results_1mil_cooling_with_annealing.pkl", "rb") as file:
        markov_results, markov_temp, markov_all_runs_data_exp = pickle.load(file)

with open("FINAL/LOG/LOG_MARKOV_NO_ADJUST_results_1mil_cooling_with_annealing.pkl", "rb") as file:
        markov_results, markov_temp, markov_all_runs_data_log = pickle.load(file)




testing between chain lengths


In [119]:

results = {"linear": markov_all_runs_data_lin , "exponential": markov_all_runs_data_exp , "logarithmic": markov_all_runs_data_log } 

iterations = 500000
markov_values = [1, 2, 5, 10, 500]



# Function to perform ANOVA and post-hoc Tukey's HSD test
def perform_anova_and_posthoc(markov_all_runs_data, method, markov_values):
    # Collect the data for ANOVA
    all_data = [] 
    group_labels = []

    for length in markov_values:
        results = markov_all_runs_data[method][length].flatten()  # Flatten 
        all_data.extend(results)  # Use extend to add the flattened array into all_data
        group_labels.extend([f"Length {length}"] * len(results))  #  labels

    # Perform One-Way ANOVA across all Markov lengths
    f_stat, p_value = stats.f_oneway(
        *[markov_all_runs_data[method][length].flatten() for length in markov_values]
    )

    print(f"ANOVA test results: F-statistic = {f_stat}, p-value = {p_value}")

    # Check if the p-value is less than 0.05 to determine significance
    if p_value < 0.05:
        print("There are significant differences among the groups. Performing post-hoc Tukey's HSD test...")

        # Convert the data into a format suitable for Tukey's HSD test
        df = pd.DataFrame({
            'value': all_data,
            'group': group_labels
        })

        # Perform Tukey's HSD test
        tukey = pairwise_tukeyhsd(df['value'], df['group'], alpha=0.05)
        print(tukey)
    else:
        print("No significant differences found in ANOVA.")

# Example usage: Run the function


for method in results: 
    print(f"\n TESTING FOR {(method).upper()} \n ")
    perform_anova_and_posthoc(results[method], method, markov_values)







 TESTING FOR LINEAR 
 
ANOVA test results: F-statistic = 782507.1002335105, p-value = 0.0
There are significant differences among the groups. Performing post-hoc Tukey's HSD test...
         Multiple Comparison of Means - Tukey HSD, FWER=0.05         
  group1    group2     meandiff  p-adj    lower       upper    reject
---------------------------------------------------------------------
 Length 1  Length 10  16853.5839   0.0  16805.2403  16901.9276   True
 Length 1   Length 2   1862.8218   0.0   1814.4782   1911.1654   True
 Length 1   Length 5   7500.3184   0.0   7451.9748   7548.6621   True
 Length 1 Length 500  26542.4494   0.0  26494.1058  26590.7931   True
Length 10   Length 2 -14990.7622   0.0 -15039.1058 -14942.4185   True
Length 10   Length 5  -9353.2655   0.0  -9401.6091  -9304.9219   True
Length 10 Length 500   9688.8655   0.0   9640.5219   9737.2091   True
 Length 2   Length 5   5637.4967   0.0    5589.153   5685.8403   True
 Length 2 Length 500  24679.6277   0.0   24631.

Testing between methods

In [118]:


# Function to perform t-tests between methods for the same Markov lengths
def perform_t_tests_across_methods(results, markov_values):
    for length in markov_values:
        # Extract data for each method for the current Markov length
        linear_data = results['linear'].get(length, None)
        exponential_data = results['exponential'].get(length, None)
        logarithmic_data = results['logarithmic'].get(length, None)

        if linear_data is None or exponential_data is None or logarithmic_data is None:
            print(f"Missing data for Markov length {length}. Skipping...")
            continue
        
        # Perform t-tests between methods for this Markov length
        print(f"Performing t-tests for Markov length {length}...\n")
        
        # Linear vs Exponential
        t_stat, p_value = stats.ttest_ind(linear_data.flatten(), exponential_data.flatten())
        print(f"Linear vs Exponential: t-statistic = {t_stat}, p-value = {p_value}")
        
        # Linear vs Logarithmic
        t_stat, p_value = stats.ttest_ind(linear_data.flatten(), logarithmic_data.flatten())
        print(f"Linear vs Logarithmic: t-statistic = {t_stat}, p-value = {p_value}")
        
        # Exponential vs Logarithmic
        t_stat, p_value = stats.ttest_ind(exponential_data.flatten(), logarithmic_data.flatten())
        print(f"Exponential vs Logarithmic: t-statistic = {t_stat}, p-value = {p_value}")
        
        print("\n")  # New line for separation between Markov lengths

# Define the Markov lengths
markov_values = [1, 2, 5, 10, 500]

# Example usage: Run the function across the three methods
perform_t_tests_across_methods({
    'linear': markov_all_runs_data_lin['linear'],   # Access the data for the 'linear' method
    'exponential': markov_all_runs_data_exp['exponential'],  # Access the data for the 'exponential' method
    'logarithmic': markov_all_runs_data_log['logarithmic']  # Access the data for the 'logarithmic' method
}, markov_values)


Performing t-tests for Markov length 1...

Linear vs Exponential: t-statistic = 177.7193727262306, p-value = 0.0
Linear vs Logarithmic: t-statistic = -858.7910351805542, p-value = 0.0
Exponential vs Logarithmic: t-statistic = -2950.4933832779366, p-value = 0.0


Performing t-tests for Markov length 2...

Linear vs Exponential: t-statistic = 265.4134438777812, p-value = 0.0
Linear vs Logarithmic: t-statistic = -537.637240420763, p-value = 0.0
Exponential vs Logarithmic: t-statistic = -2545.8479850538315, p-value = 0.0


Performing t-tests for Markov length 5...

Linear vs Exponential: t-statistic = 489.99751303495174, p-value = 0.0
Linear vs Logarithmic: t-statistic = -116.9072057410292, p-value = 0.0
Exponential vs Logarithmic: t-statistic = -1924.6030295489443, p-value = 0.0


Performing t-tests for Markov length 10...

Linear vs Exponential: t-statistic = 1033.7948286569142, p-value = 0.0
Linear vs Logarithmic: t-statistic = 431.03688954236094, p-value = 0.0
Exponential vs Logarithmi